In [1]:
from lxml import html
import requests
from string import ascii_lowercase
import sys
import pickle
from xml.etree.ElementTree import tostring
from IPython.display import clear_output
from time import sleep

# base URL
page_root_url = 'http://www.jokebuddha.com/'
# file name for the topics
topics_filename = 'topics.p'
# filename for the jokes
jokes_filename = 'jokes.p'

In [2]:
# gather topics
page_topic_base_url = page_root_url + 'topics/'
topics = set([])
for i in ascii_lowercase :
    page_topic_url = page_topic_base_url + i.upper()
    page_response = requests.get(page_topic_url)
    tree = html.fromstring(page_response.content)
    all_topics_for_letter = tree.xpath('//div[@class="topicsList"]//text()')
    for topic_for_letter in all_topics_for_letter:
        if len(topic_for_letter) > 1:
            topics.add(topic_for_letter)
            print(page_topic_url + ': ' + topic_for_letter)
            clear_output(wait=True)
topics = sorted(topics)
topics = topics[1:]

with open(topics_filename, 'wb') as fp:
    pickle.dump(topics, fp)
    
print('Gathered a total of ' + str(len(topics)) + ' topics')

Gathered a total of 3851 topics


In [19]:
def get_jokes(base_page_url, iter_, topic, jokes, first_joke, tot_n_jokes = 0):  
    MAX_JOKE_LENGTH = 200
    no_more_jokes = True
    
    final_page_url = base_page_url + str(iter_)
    got_request = False
    while not got_request:
        try:
            page_response = requests.get(final_page_url, stream=True)
            got_request = True
        except:
            print('Taking a nap until the website responds zzzZZZ...')
            sleep(5)
    
    tree = html.fromstring(page_response.content)    
    joke_html_texts = tree.xpath('//p[@class="joke-inner"]')
    if len(joke_html_texts) == 0:
        return no_more_jokes, jokes, tot_n_jokes
    
    for joke_html_text in joke_html_texts:
        joke_text_list = [x for x in joke_html_text.itertext()]        
        if len(joke_text_list) == 0:
            return no_more_jokes, jokes, tot_n_jokes
        formatted_joke = ' '.join(joke_text_list)
        if formatted_joke == first_joke:
            return no_more_jokes, jokes, tot_n_jokes        
        if not formatted_joke.find('Read more') == 0 and len(formatted_joke) <= MAX_JOKE_LENGTH:
            jokes[topic].append(formatted_joke)
            print('Total number of jokes: ' + str(tot_n_jokes))
            print('Topic ' + topic + ': ' + formatted_joke)
            clear_output(wait=True)
            tot_n_jokes += 1
    
    no_more_jokes = False
    return no_more_jokes, jokes, tot_n_jokes

def get_topic_jokes(topic,page_root_url,jokes, tot_n_jokes = 0):
    final_page_url = page_root_url + topic + '/recent/'
    jokes[topic] = []
    first_joke = 'NotAJoke'
    no_more_jokes, jokes, tot_n_jokes = get_jokes(final_page_url, 1, topic, jokes, first_joke, tot_n_jokes)
    if no_more_jokes:
        return jokes, no_more_jokes, tot_n_jokes
    if len(jokes[topic]) > 0:
        first_joke = jokes[topic][0]

    for i in range(100):  
        no_more_jokes, jokes, tot_n_jokes = get_jokes(final_page_url, i+2, topic, jokes, first_joke, tot_n_jokes)   
        if no_more_jokes:            
            break    
    return jokes, no_more_jokes, tot_n_jokes       

In [20]:
# if the joke scraping was interrupted, run this cell
topics = pickle.load( open( topics_filename, "rb" ) ) 

jokes = {}
tot_n_jokes = 0
curr_topic = topics[0]
try:
    jokes = pickle.load( open(jokes_filename, "rb") )    
    topics_without_jokes = []
    for topic in topics:
        n_jokes_topic = 0
        if topic in jokes.keys():        
            jokes_topic = jokes[topic]
            for joke in jokes_topic:
                n_jokes_topic+=1
        else:
            topics_without_jokes.append(topic)
        tot_n_jokes += n_jokes_topic

    jokes_topics = sorted(jokes.keys())
    curr_topic = jokes_topics[-1]
    print('Last processed topic before interruption was: ' + curr_topic)
except:
    print('Could not find file ' + jokes_filename + '. Starting from scratch.')
    
jokes[curr_topic] = []
topics_to_process = topics[topics.index(curr_topic):]

for topic in topics_to_process:
    curr_topic = topic
    jokes, no_more_jokes, tot_n_jokes = get_topic_jokes(topic,page_root_url,jokes,tot_n_jokes)
    with open(jokes_filename, 'w+b') as fp:
        pickle.dump(jokes, fp)
    
jokes = pickle.load( open(jokes_filename, "rb") )
    


Last processed topic before interruption was: toy
Total number of jokes: 42734
 Rohit: Easily, If One Is A Toyephant Go In A Fridge/


In [63]:
topics = pickle.load( open( topics_filename, "rb" ) )   
jokes = pickle.load( open( jokes_filename, "rb" ) )

tot_n_jokes = 0
topics_without_jokes = []
for topic in topics:
    if topic in jokes.keys():
        n_jokes_topic = 0
        jokes_topic = jokes[topic]
        for joke in jokes_topic:
            n_jokes_topic+=1
    else:
        topics_without_jokes.append(topic)
    tot_n_jokes += n_jokes_topic

In [59]:
print(tot_n_jokes)

485


In [ ]:
with open('curr_topic', 'wb') as fp:
    print('')